In [1]:
import pandas as pd
import xgboost as xgb

data = pd.DataFrame()



SmartSHARK

In [22]:
import pandas as pd

data = pd.read_csv('model_data_TimeAware_compress.csv')
projects = data.project.unique()
projectVersionMap = {}
for project in projects:
    projectParts = project.split("_")
    groupName = projectParts[0] + "_" + projectParts[1]
    if groupName not in projectVersionMap:
        projectVersionMap[groupName] = []
    projectVersionMap[groupName].append(project)

projectVersionMap

{'smartshark_ant-ivy': ['smartshark_ant-ivy_1.4.1',
  'smartshark_ant-ivy_2.0.0',
  'smartshark_ant-ivy_2.1.0',
  'smartshark_ant-ivy_2.2.0',
  'smartshark_ant-ivy_2.3.0',
  'smartshark_ant-ivy_2.4.0'],
 'smartshark_archiva': ['smartshark_archiva_1.0',
  'smartshark_archiva_1.1',
  'smartshark_archiva_1.2',
  'smartshark_archiva_1.3',
  'smartshark_archiva_2.0.0',
  'smartshark_archiva_2.1.0',
  'smartshark_archiva_2.2.0'],
 'smartshark_calcite': ['smartshark_calcite_1.0.0',
  'smartshark_calcite_1.1.0',
  'smartshark_calcite_1.10.0',
  'smartshark_calcite_1.11.0',
  'smartshark_calcite_1.12.0',
  'smartshark_calcite_1.13.0',
  'smartshark_calcite_1.14.0',
  'smartshark_calcite_1.15.0',
  'smartshark_calcite_1.2.0',
  'smartshark_calcite_1.3.0',
  'smartshark_calcite_1.4.0',
  'smartshark_calcite_1.5.0',
  'smartshark_calcite_1.6.0',
  'smartshark_calcite_1.7.0',
  'smartshark_calcite_1.8.0',
  'smartshark_calcite_1.9.0'],
 'smartshark_cayenne': ['smartshark_cayenne_3.0.0',
  'smartsha

In [19]:
for key in projectVersionMap:
    subData = data[data['project'].isin(projectVersionMap[key])]
    subData.to_csv('data_TimeAware/' + key +'.csv')
    subData2 = data[~data['project'].isin(projectVersionMap[key])]
    subData2.to_csv('data_TimeAware/' + key +'_train.csv')

In [27]:
import os
import pandas as pd

data_mp = pd.read_csv('statistic_smartshark.csv')


data = pd.DataFrame()
for file in os.listdir("smartshark_other/amalgam"):
    project, version = file.replace('smartshark_','').replace('_output.txt','').split("_")
    element = data_mp.loc[(data_mp['Project'] == "smartshark_" + project) & (data_mp['Version'] == version), 'Bugs'].values[0]
    data = data.append({"project" : project, "version" : version, "bugs": element}, ignore_index=True)

agg_funcs = {'bugs' : 'sum', 
         'version' : 'count'
}

data = data.groupby("project").agg(agg_funcs)

In [28]:
import numpy as np 

def evaluate_for_key(key, bugs_length=None):
    resultData = pd.read_csv('results_TimeAware/' + key + '.csv')  
    resultData['project_bugs'] = resultData['project'] + resultData['bug'].astype(str)
    bugs = resultData.project_bugs.unique()
    print("Project " + key)
    if bugs_length == None:
        bugs_length = len(bugs)
    print(bugs_length)
    count1 = 0;
    count5 = 0;
    count10 = 0;
    mapValue = 0;
    mrr = 0;
    for bug in bugs:
        subData = resultData[resultData['project_bugs'] == bug]
        subDataSorted = subData.sort_values('Result2',ascending=False)
        
        if subDataSorted.iloc[0,-3] == 1:
            count1 += 1;
        if 1 in subDataSorted.values[0:5,-3]:
            count5 += 1;
        if 1 in subDataSorted.values[0:10,-3]:
            count10 += 1;
        
        summe = 0;
        rr = 0;
        retrieved_d = 0;
        i = 0
        c = 0
        for resultV in subDataSorted.values[:10,-3]:
            if resultV == 1:
                c += 1
                retrieved_d += 1;
                summe += (retrieved_d /(i+1))
            if resultV == 1 and rr == 0:    
                rr += (1/ (i+1))
            i += 1
        if c != 0:
            mapValue += (summe/c)
        else: 
            print("c zero!" + str(bug))
            mapValue +=0
            #rr = 1
        mrr += rr
    return count1/bugs_length,count5/bugs_length,count10/bugs_length,mapValue/bugs_length,mrr/bugs_length

In [29]:
maps = []
mrrs = []
top1s = []
top5s = []
top10s = []

data = data.reset_index()
print(data.columns)
    
gbm = xgb.Booster(model_file='model/smartshark2.json')
for key in projectVersionMap:
    print(key)
    test = pd.read_csv('data_TimeAware/' + key +'.csv')    
    columns = test.drop(['Unnamed: 0','project','bug','Result','description_matching'], axis=1).columns
    result = pd.DataFrame(test)
    result['Result2'] = gbm.predict(xgb.DMatrix(test[columns]))
    result.to_csv('results_TimeAware/' + key +'.csv', index=False, sep=',')
    
    pro = key.replace('smartshark_','')
    bugs_length = data[data["project"] == pro]["bugs"].values[0]
    count1, count5, count10, mapValue, mrr = evaluate_for_key(key, bugs_length)
    print("Count 1: " + str(count1) + " Count 5: " + str(count5) + " Count 10:" + str(count10))
    print("MAP " + str(mapValue))
    print("MRR " + str(mrr))
        
    top1s.append(count1)
    top5s.append(count5)
    top10s.append(count10)
    maps.append(mapValue)
    mrrs.append(mrr)
    

Index(['project', 'bugs', 'version'], dtype='object')
smartshark_ant-ivy
Project smartshark_ant-ivy
374.0
c zero!smartshark_ant-ivy_1.4.11000038
c zero!smartshark_ant-ivy_1.4.110000100
c zero!smartshark_ant-ivy_1.4.13000017
c zero!smartshark_ant-ivy_1.4.13000019
c zero!smartshark_ant-ivy_1.4.13000024
c zero!smartshark_ant-ivy_1.4.13000027
c zero!smartshark_ant-ivy_1.4.13000028
c zero!smartshark_ant-ivy_1.4.13000048
c zero!smartshark_ant-ivy_1.4.13000052
c zero!smartshark_ant-ivy_1.4.13000054
c zero!smartshark_ant-ivy_1.4.13000057
c zero!smartshark_ant-ivy_1.4.13000061
c zero!smartshark_ant-ivy_1.4.13000065
c zero!smartshark_ant-ivy_1.4.13000074
c zero!smartshark_ant-ivy_2.0.0100008
c zero!smartshark_ant-ivy_2.0.0100009
c zero!smartshark_ant-ivy_2.0.01000014
c zero!smartshark_ant-ivy_2.0.01000016
c zero!smartshark_ant-ivy_2.0.01000026
c zero!smartshark_ant-ivy_2.0.01000028
c zero!smartshark_ant-ivy_2.0.01000037
c zero!smartshark_ant-ivy_2.0.01000054
c zero!smartshark_ant-ivy_2.0.0100005

c zero!smartshark_cayenne_3.0.01000017
c zero!smartshark_cayenne_3.0.01000018
c zero!smartshark_cayenne_3.0.01000019
c zero!smartshark_cayenne_3.0.01000023
c zero!smartshark_cayenne_3.0.01000033
c zero!smartshark_cayenne_3.0.01000035
c zero!smartshark_cayenne_3.0.01000037
c zero!smartshark_cayenne_3.0.01000039
c zero!smartshark_cayenne_3.0.01000041
c zero!smartshark_cayenne_3.0.01000047
c zero!smartshark_cayenne_3.0.01000049
c zero!smartshark_cayenne_3.0.01000051
c zero!smartshark_cayenne_3.0.01000057
c zero!smartshark_cayenne_3.0.01000058
c zero!smartshark_cayenne_3.0.01000061
c zero!smartshark_cayenne_3.0.01000064
c zero!smartshark_cayenne_3.0.01000067
c zero!smartshark_cayenne_3.0.01000072
c zero!smartshark_cayenne_3.0.01000075
c zero!smartshark_cayenne_3.0.01000077
c zero!smartshark_cayenne_3.0.01000080
c zero!smartshark_cayenne_3.0.01000087
c zero!smartshark_cayenne_3.0.01000090
c zero!smartshark_cayenne_3.0.01000094
c zero!smartshark_cayenne_3.0.01000095
c zero!smartshark_cayenne

c zero!smartshark_commons-math_2.26000028
c zero!smartshark_commons-math_2.26000030
c zero!smartshark_commons-math_2.26000034
c zero!smartshark_commons-math_2.26000043
c zero!smartshark_commons-math_2.26000046
c zero!smartshark_commons-math_2.26000053
c zero!smartshark_commons-math_2.2800003
c zero!smartshark_commons-math_3.2900004
c zero!smartshark_commons-math_3.2900005
c zero!smartshark_commons-math_3.2900008
c zero!smartshark_commons-math_3.29000014
c zero!smartshark_commons-math_3.29000022
c zero!smartshark_commons-math_3.411000011
c zero!smartshark_commons-math_3.411000028
c zero!smartshark_commons-math_3.51200004
c zero!smartshark_commons-math_3.51200006
c zero!smartshark_commons-math_3.51200008
Count 1: 0.6029411764705882 Count 5: 0.8235294117647058 Count 10:0.8921568627450981
MAP 0.6232132158418923
MRR 0.701100995953937
smartshark_commons-net
Project smartshark_commons-net
117.0
c zero!smartshark_commons-net_3.1700005
c zero!smartshark_commons-net_3.1700006
c zero!smartshark_c

c zero!smartshark_jspwiki_2.8.09000079
c zero!smartshark_jspwiki_2.8.09000081
c zero!smartshark_jspwiki_2.8.09000082
c zero!smartshark_jspwiki_2.8.09000084
c zero!smartshark_jspwiki_2.8.09000086
c zero!smartshark_jspwiki_2.8.09000089
c zero!smartshark_jspwiki_2.8.09000090
c zero!smartshark_jspwiki_2.8.09000092
c zero!smartshark_jspwiki_2.8.09000093
c zero!smartshark_jspwiki_2.8.09000094
c zero!smartshark_jspwiki_2.8.09000095
c zero!smartshark_jspwiki_2.9.09000011
c zero!smartshark_jspwiki_2.9.09000013
c zero!smartshark_jspwiki_2.9.09000015
c zero!smartshark_jspwiki_2.9.09000016
c zero!smartshark_jspwiki_2.9.09000018
c zero!smartshark_jspwiki_2.9.09000023
c zero!smartshark_jspwiki_2.9.09000025
c zero!smartshark_jspwiki_2.9.09000026
c zero!smartshark_jspwiki_2.9.09000034
c zero!smartshark_jspwiki_2.9.09000039
c zero!smartshark_jspwiki_2.9.09000093
Count 1: 0.2773109243697479 Count 5: 0.4369747899159664 Count 10:0.5630252100840336
MAP 0.32597539015606253
MRR 0.34987995198079236
smartshark

Project smartshark_mahout
222.0
c zero!smartshark_mahout_0.10.09000031
c zero!smartshark_mahout_0.10.09000037
c zero!smartshark_mahout_0.10.09000041
c zero!smartshark_mahout_0.10.09000046
c zero!smartshark_mahout_0.10.09000047
c zero!smartshark_mahout_0.10.09000050
c zero!smartshark_mahout_0.10.09000058
c zero!smartshark_mahout_0.10.09000061
c zero!smartshark_mahout_0.10.09000067
c zero!smartshark_mahout_0.11.0300002
c zero!smartshark_mahout_0.11.03000015
c zero!smartshark_mahout_0.11.03000016
c zero!smartshark_mahout_0.11.03000019
c zero!smartshark_mahout_0.11.03000021
c zero!smartshark_mahout_0.12.0200001
c zero!smartshark_mahout_0.19000062
c zero!smartshark_mahout_0.3600004
c zero!smartshark_mahout_0.38000011
c zero!smartshark_mahout_0.38000019
c zero!smartshark_mahout_0.48000043
c zero!smartshark_mahout_0.5800002
c zero!smartshark_mahout_0.5800004
c zero!smartshark_mahout_0.5800005
c zero!smartshark_mahout_0.5800006
c zero!smartshark_mahout_0.58000011
c zero!smartshark_mahout_0.580

c zero!smartshark_systemml_0.10600008
c zero!smartshark_systemml_0.106000015
c zero!smartshark_systemml_0.106000016
c zero!smartshark_systemml_0.106000017
c zero!smartshark_systemml_0.106000019
c zero!smartshark_systemml_0.106000020
c zero!smartshark_systemml_0.106000021
c zero!smartshark_systemml_0.11400001
c zero!smartshark_systemml_0.11400004
c zero!smartshark_systemml_0.11400006
c zero!smartshark_systemml_0.11400008
c zero!smartshark_systemml_0.114000011
c zero!smartshark_systemml_0.114000017
c zero!smartshark_systemml_0.114000018
c zero!smartshark_systemml_0.114000019
c zero!smartshark_systemml_0.114000023
c zero!smartshark_systemml_0.114000025
c zero!smartshark_systemml_0.114000026
c zero!smartshark_systemml_0.114000028
c zero!smartshark_systemml_0.114000037
c zero!smartshark_systemml_0.12500001
c zero!smartshark_systemml_0.13200001
c zero!smartshark_systemml_0.13200003
c zero!smartshark_systemml_0.13200004
c zero!smartshark_systemml_0.13200005
c zero!smartshark_systemml_0.132000

In [30]:
import statistics 

print("Average MAP: " + str(statistics.median(maps)))
print("Average MRR: " + str(statistics.median(mrrs)))
print("Average 1: " + str(statistics.mean(top1s)))
print("Average 5: " + str(statistics.mean(top5s)))
print("Average 10: " + str(statistics.mean(top10s)))

Average MAP: 0.5474729685452162
Average MRR: 0.5966935596427021
Average 1: 0.48488561419573756
Average 5: 0.749743127026261
Average 10: 0.8388953816725183


In [55]:
sum(maps)/sum(data['bugs'])
Average MAP: 0.5821443800289604
Average MRR: 0.7551110616559689
Average 1: 0.6435088540648778
Average 5: 0.8512669335925249
Average 10: 0.89962887419386

SyntaxError: invalid syntax (<ipython-input-55-738d9ca4552e>, line 2)

In [32]:
mrrs

[0.499087513793396,
 0.42686915887850474,
 0.5576836290071584,
 0.40089203725567363,
 0.34732142857142856,
 0.7420634920634921,
 0.8366666666666666,
 0.7778745644599303,
 0.7230158730158731,
 0.7251871209188283,
 0.6556593406593407,
 0.6666666666666666,
 0.8981481481481481,
 0.6356481481481482,
 0.5394973544973544,
 0.83513986013986,
 0.701100995953937,
 0.8245217745217744,
 0.7053921568627451,
 0.7570017636684303,
 0.626384164777022,
 0.4827649986548293,
 0.31840476190476186,
 0.5610091743119265,
 0.6591425459072517,
 0.34987995198079236,
 0.4466429851510499,
 0.4288402457757297,
 0.4107531055900622,
 0.68264157014157,
 0.5472347976749385,
 0.5836689463019251,
 0.5378808611567233,
 0.6097181729834791,
 0.6720027017899358,
 0.3465593084779131,
 0.5734672619047617,
 0.5660211640211642]